# Jupyter File Drag & Drop Widget - Demo

This notebook demonstrates how to use the drag-and-drop file upload widget for JupyterLab.

**Supported file formats:**
- CSV (`.csv`)
- Excel (`.xlsx`, `.xlsm`, `.xls`) - with multi-sheet support
- Feather (`.feather`)
- Parquet (`.parquet`)

## 1. FileDrop: Quick Start

The `FileDrop` class provides a simple one-line API for drag-and-drop file uploads.

In [1]:
import sys
sys.path.insert(0, '..')  # Add parent directory to find the module

from ipyfiledrop import FileDrop

# One-line creation with named drop zones
fd = FileDrop("Dataset A", "Dataset B")
fd.display()

<IPython.core.display.Javascript object>

FileDrop(labels=['Dataset A', 'Dataset B'], loaded=[])

In [ ]:
# Access loaded DataFrames
print("Loaded datasets:", list(fd.datasets.keys()))

# Access individual DataFrame (returns selected sheet for Excel files)
df_a = fd["Dataset A"]  # Returns DataFrame or None
if df_a is not None:
    print(f"Dataset A shape: {df_a.shape}")
    display(df_a.head())

## 2. Multi-Sheet Excel Support

When you drop an Excel file with multiple sheets, a **dropdown selector** appears to switch between sheets.

**Try it:** Drop an Excel file with multiple sheets to see the dropdown appear.

In [ ]:
# Create a widget for Excel files
fd_excel = FileDrop("Excel Data")
fd_excel.display()

In [ ]:
fd_excel['Excel Data']

In [ ]:
# After dropping a multi-sheet Excel file:

# Get the currently selected DataFrame
df = fd_excel["Excel Data"]
if df is not None:
    print(f"Selected sheet shape: {df.shape}")

# Get ALL sheets as a dictionary
all_sheets = fd_excel.get_all_sheets("Excel Data")
if all_sheets:
    print(f"\nAvailable sheets: {list(all_sheets.keys())}")
    for name, sheet_df in all_sheets.items():
        print(f"  - {name}: {sheet_df.shape[0]} rows × {sheet_df.shape[1]} columns")

In [ ]:
# Programmatically select a different sheet
# (This also updates the dropdown in the widget)

if fd_excel.get_all_sheets("Excel Data"):
    sheets = list(fd_excel.get_all_sheets("Excel Data").keys())
    if len(sheets) > 1:
        fd_excel.select_sheet("Excel Data", sheets[1])  # Select second sheet
        print(f"Selected sheet: {sheets[1]}")

## 3. All Supported File Formats

The widget supports CSV, Excel (xlsx/xlsm/xls), Feather, and Parquet files.

In [ ]:
# Create drop zones for different file types
fd_formats = FileDrop("CSV", "Excel", "Feather", "Parquet")
fd_formats.display()

In [ ]:
# Check what's loaded
for label in ["CSV", "Excel", "Feather", "Parquet"]:
    df = fd_formats[label]
    if df is not None:
        print(f"{label}: {df.shape[0]} rows × {df.shape[1]} columns")
    else:
        print(f"{label}: (no file loaded)")

## 4. Dynamic Drop Zone Management

Add or remove drop zones dynamically.

In [ ]:
fd_dynamic = FileDrop("Initial")
fd_dynamic.display()

In [ ]:
# Add new drop zones (method chaining supported)
fd_dynamic.add("Added 1").add("Added 2")
print(fd_dynamic)

In [ ]:
# Remove a drop zone
fd_dynamic.remove("Added 1")
print(fd_dynamic)

## 5. Embedding in ipywidgets Containers

Use the `.ui` property to embed FileDrop in Accordion, Tab, VBox, etc.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Embedding in Accordion
fd_acc1 = FileDrop("Training", "Validation")
fd_acc2 = FileDrop("Test")

accordion = widgets.Accordion(children=[fd_acc1.ui, fd_acc2.ui])
accordion.set_title(0, "Train/Val Data")
accordion.set_title(1, "Test Data")

display(accordion)

In [ ]:
# Embedding in Tab
fd_tab1 = FileDrop("CSV Files")
fd_tab2 = FileDrop("Excel Files")

tab = widgets.Tab(children=[fd_tab1.ui, fd_tab2.ui])
tab.set_title(0, "CSV")
tab.set_title(1, "Excel")

display(tab)

In [ ]:
# Embedding with a button and output
fd_btn = FileDrop("Upload")
btn = widgets.Button(description="Process Data", button_style="primary")
output = widgets.Output()

def on_click(b):
    with output:
        output.clear_output()
        df = fd_btn["Upload"]
        if df is not None:
            print(f"Processing {df.shape[0]} rows...")
            display(df.describe())
        else:
            print("No file uploaded yet!")

btn.on_click(on_click)

display(widgets.VBox([fd_btn.ui, btn, output]))

## 6. IFrameDropWidget: Low-Level API

For more control, use `IFrameDropWidget` directly with the `on_data_ready` callback.

In [ ]:
from ipyfiledrop import IFrameDropWidget

# Install global listener (FileDrop does this automatically)
IFrameDropWidget.install_global_listener()

In [ ]:
# Define callback - receives Dict[str, DataFrame]
def on_data_ready(filename, data):
    """Called when a file is loaded.
    
    Args:
        filename: Name of the uploaded file
        data: Dict[str, DataFrame] - keys are sheet names for Excel, 'data' for others
    """
    print(f"\nLoaded: {filename}")
    print(f"Sheets/Keys: {list(data.keys())}")
    for name, df in data.items():
        print(f"  {name}: {df.shape[0]} rows × {df.shape[1]} columns")

# Create widget with callback
widget = IFrameDropWidget(on_data_ready=on_data_ready)
widget.display()

In [ ]:
# Access data via properties
if widget.data:
    print(f"Available sheets: {widget.sheet_names}")
    print(f"Currently selected: {widget.selected_key}")
    print(f"\nSelected DataFrame:")
    display(widget.selected_dataframe.head())
else:
    print("No data loaded yet. Drop a file in the widget above!")

## 7. Full datasets Property

The `datasets` property returns all loaded data with full metadata.

In [ ]:
# Create and load some data
fd_meta = FileDrop("Data 1", "Data 2")
fd_meta.display()

In [ ]:
# After loading files, inspect the datasets property
for label, info in fd_meta.datasets.items():
    print(f"\n{label}:")
    print(f"  Filename: {info['filename']}")
    print(f"  Selected: {info['selected']}")
    print(f"  Available sheets: {list(info['data'].keys())}")